In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [86]:
p = np.array([4, 2, 2, 1]).reshape(-1, 1)
d = np.array([20, 5, 10, 15]).reshape(-1, 1)
s = np.array([30, 10, 5, 0]).reshape(-1, 1)
dtilde = np.array([10, 25, 5, 15]).reshape(-1, 1)
stilde = np.array([5, 20, 15, 20]).reshape(-1, 1)

k = 0.5

def solve_problem(set_t_zero):
  
  t = cp.Variable((4, 1))
  B = cp.Variable((4, 4), nonneg = True)
  Btilde = cp.Variable((4, 4), nonneg = True)
  ones = np.ones((4, 1))

  obj1= k * cp.norm1(t)
  obj2 = p.T @ B.T @  ones 
  obj3 = p.T @ Btilde.T @ ones

  obj = cp.Minimize(obj1 + obj2 + obj3)

  not_allowed = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 1), (2, 3)]
                          
  constraints = [
      B @ ones == d,
      Btilde @ ones == dtilde,
      B.T @ ones + t <= s,
      Btilde.T @ ones -t <= stilde
  ]

  for (i, j) in not_allowed:
    constraints += [ B[i, j] == 0, Btilde[i, j] == 0]

  if set_t_zero:
    constraints += [t[:, -1] == 0]

  prob = cp.Problem(obj, constraints)
  prob.solve()

  if prob.status == 'optimal':
    B = np.round(B.value, 2)
    Btilde = np.round(Btilde.value, 2)
    t = np.round(t.value, 2)

    print(f"B:\n{B}")
    print(f"B tilde:\n{Btilde}")
    print(f"t:\n{t}")

    print(f"Shipment cost:{obj1.value}")
    print(f"Minimum cost:{obj.value}")
  else:
    print("Problem not optimal")

solve_problem(False)

B:
[[20.    0.    0.    0.  ]
 [ 1.34  3.66  0.    0.  ]
 [ 1.37  0.    8.63  0.  ]
 [ 1.25  2.38  3.23  8.14]]
B tilde:
[[10.    0.    0.    0.  ]
 [ 1.04 23.96  0.    0.  ]
 [ 0.    0.    5.    0.  ]
 [ 0.    0.    3.14 11.86]]
t:
[[ 6.04]
 [ 3.96]
 [-6.86]
 [-8.14]]
Shipment cost:12.50000000646607
Minimum cost:272.49999999737315


In [85]:
solve_problem(True)

Problem not optimal
